In [3]:
import os
import sys

module_path = os.path.abspath("/Users/a1132077/development/")
if module_path not in sys.path:
    sys.path.append(module_path)
from phenoct.src import phenoct as pct

tube = pct.Tube(
    "///Volumes/phenoxray/Result/Ears/xray/0674-Tucker/SH0041/XRAYCT_V_SH0041_221212144355987_ears_1_200mm_084mu_0674-Tucker_221212182054853/XRAYCT_V_X-005246-03_221212144355987_ears_1_200mm_084mu_0674-Tucker-SH0041_221215215629878.rek")


In [4]:
# tube.segment_sample_holder(start_slice=1550, stop_slice=1600, debug=False, attenuation_threshold = 5000)
#
#
# tube.segment_sample_holder(start_slice=1550, stop_slice=1600, debug=False, attenuation_threshold = [])


tube.segment_sample_holder(start_slice=1550, stop_slice=1600, debug=False)


Segmenting slice: 1599: 100%|██████████| 50/50 [00:00<00:00, 70.99it/s]


In [2]:
#a.crop_segmented()

In [3]:
# higher attenuation threshold and fewer slices for testing. This is slow.
tube.segment_sample_holder(start_slice=1550, stop_slice=1600, debug=False)

tube.watershed_seeds()

tube.view_segmented_data()


#tube.write_colourised_tiff('a_cc.tiff')

Segmenting slice: 1599: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 90.00it/s]


In [5]:
tube.create_animation('tube.mp4')


/Users/a1132077/opt/anaconda3/envs/datasci310/lib/python3.10/site-packages/napari/_vispy/layers/image.py:231: UserWarning: data shape (2760, 438, 392) exceeds GL_MAX_TEXTURE_SIZE 2048 in at least one axis and will be downsampled. Rendering is currently in 3D mode.
  warnings.warn(


Rendering frames...


100%|██████████| 91/91 [00:03<00:00, 27.98it/s]


In [4]:
#tube.create_animation("/Users/a1132077/Downloads/mia_test.mp4")